In [24]:
import sympy as sy
import numpy as np
import casadi as ca
import pathlib
from SE3Lie import *
import numpy as np
from sympy import *
from pyecca.lie import so3, r3, se3
from pyecca.lie.util import DirectProduct
from pyecca.lie.so3 import Quat, Dcm, Euler, Mrp
from pyecca.lie.r3 import R3
from pyecca.lie.se3 import SE3
from pyecca.test import test_lie
import casadi as ca
from SE3Lie import se3, SE3, se3_diff_correction_inv, dot_plot_draw
from casadi.tools.graph import graph, dotdraw, dotsave
import os
import pydot
from Sympy_To_Casadi import SympyToCasadi
from matplotlib import pyplot as plt

Sympy for U Matrix of SE3 for Reference 

In [3]:
x, y, z, theta_1, theta_2, theta_3 = symbols("x,y,z,theta_1,theta_2,theta_3", real=True)
theta = symbols("theta", real=True)
ad = sy.Matrix(
    [
        [0, -theta_3, theta_2, 0, -z, y],
        [theta_3, 0, -theta_1, z, 0, -x],
        [-theta_2, theta_1, 0, -y, x, 0],
        [0, 0, 0, 0, -theta_3, theta_2],
        [0, 0, 0, theta_3, 0, -theta_1],
        [0, 0, 0, -theta_2, theta_1, 0],
    ]
)
I = eye(6)
A, B = symbols("A, B", real=True)
UInv = I + A * ad + B * ad**2  # separate into Odd and Even terms
det_uInv = det(UInv)
Uadj = UInv.adjugate()
u_mat = Uadj / det_uInv
u_mat

Matrix([
[((A*theta_1 - B*theta_2*theta_3)*(A*theta_1 + B*theta_2*theta_3) + (-B*(-theta_1**2 - theta_2**2) - 1)*(-B*(-theta_1**2 - theta_3**2) - 1))*(-(-A*theta_2 - B*theta_1*theta_3)*((A*theta_1 + B*theta_2*theta_3)*(A*theta_3 + B*theta_1*theta_2) + (-A*theta_2 + B*theta_1*theta_3)*(B*(-theta_1**2 - theta_2**2) + 1)) - (A*theta_3 - B*theta_1*theta_2)*((-A*theta_1 + B*theta_2*theta_3)*(-A*theta_2 + B*theta_1*theta_3) + (A*theta_3 + B*theta_1*theta_2)*(B*(-theta_1**2 - theta_3**2) + 1)) - (-B*(-theta_2**2 - theta_3**2) - 1)*((A*theta_1 - B*theta_2*theta_3)*(A*theta_1 + B*theta_2*theta_3) + (-B*(-theta_1**2 - theta_2**2) - 1)*(-B*(-theta_1**2 - theta_3**2) - 1)) - ((-A*theta_2 - B*theta_1*theta_3)*(-A*theta_2 + B*theta_1*theta_3) + (A*theta_3 - B*theta_1*theta_2)*(A*theta_3 + B*theta_1*theta_2))*(-B*(-theta_1**2 - theta_2**2) - B*(-theta_1**2 - theta_3**2) - 2))/(A**2*theta_1**2 + A**2*theta_2**2 + A**2*theta_3**2 + B**2*theta_1**4 + 2*B**2*theta_1**2*theta_2**2 + 2*B**2*theta_1**2*thet

In [31]:
k = symbols("k", real=True)
ASum = sy.summation(((-1) ** k) * theta ** (2 * k) / (factorial(2 * k + 2)), (k, 0, oo))
# Asub = simplify(ASum) #A term
BSum = summation(((-1) ** k) * theta ** (2 * k) / (factorial(2 * k + 3)), (k, 0, oo))
A_series = series(ASum, theta)
B_series = series(BSum, theta)

u_mat = u_mat.applyfunc(lambda x: x.subs({A: A_series, B: B_series}))
# subTerms = {theta_2**2 + theta_3**2: theta**2 - theta_1**2, theta_1**2 + theta_3**2: theta**2 - theta_2**2, theta_1**2 + theta_2**2: theta**2 - theta_3**2}
# UInv = UInv.applyfunc(lambda x: x.subs(subTerms).simplify())
# u_mat #U inverse with A and B terms substituted closed form

In [32]:
u_mat
# plt.imshow(u_mat, interpolation='nearest')

Matrix([
[                                             ((theta_1*(1/2 - theta**2/24 + theta**4/720 + O(theta**6)) - theta_2*theta_3*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)))*(theta_1*(1/2 - theta**2/24 + theta**4/720 + O(theta**6)) + theta_2*theta_3*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6))) + (-(-theta_1**2 - theta_2**2)*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)) - 1)*(-(-theta_1**2 - theta_3**2)*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)) - 1))*(-(-(-theta_2**2 - theta_3**2)*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)) - 1)*((theta_1*(1/2 - theta**2/24 + theta**4/720 + O(theta**6)) - theta_2*theta_3*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)))*(theta_1*(1/2 - theta**2/24 + theta**4/720 + O(theta**6)) + theta_2*theta_3*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6))) + (-(-theta_1**2 - theta_2**2)*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)) - 1)*(-(-theta_1**2 - theta_3**2)*(1/6 - theta**2/120 + theta**4/5040 + O(theta**6)) - 1)

U Inv matrix in casadi form and dot draw plot


parser = SympyToCasadi()
UInv_ca = parser.parse(UInv, cse=True)
UInv_ca

In [ ]:
# error because parser inputs are in series
# todo
parser = SympyToCasadi()
uMat_ca = parser.parse(u_mat, cse=True)
uMat_ca

In [22]:
# reference example for Sympy_To_Casadi
x = sy.symbols("x")
A = sy.Matrix(
    [
        [sy.sin(x), 2 * x**2, 4 * x**2 * sy.sin(x)],
        [1 + x, x * sy.cos(x), 2 * x**2 + 3 / x],
        [2 - x, x, sy.sqrt(x)],
    ]
)
A
parser = SympyToCasadi()
f_ca = parser.parse(A)
f_ca

SX(@1=2, 
[[sin(x), (@1*sq(x)), ((4*sq(x))*sin(x))], 
 [(1+x), (x*cos(x)), ((@1*sq(x))+(3/x))], 
 [(@1-x), x, sqrt(x)]])

In [38]:
# dot_plot_draw(UInv_ca)